In [1]:
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
# add partisan lean and get US state info from location

# dataframe containing state name, state abbrevation, and partisan lean for each US state
lean_df = pd.read_csv('partisan_lean.csv')


def get_partisan_lean(user_location):
    count = 0
    user_lean = -1
    state = ''
    
    # loop through states
    for index, row in lean_df.iterrows():
        # there's a couple nans in the dataset
        if type(user_location) != type('a'):
            continue
        # check if abbreviation is in string
        if ' ' + row['abbrev'] in user_location:
            user_lean = row['vote_share']
            count += 1
            state = row['state'].title()
        
        # check if state name is in string
        if row['state'] in user_location.upper():
            # very rarely a city will have the same name as a state, this prioritizes using the abbreviation
            if count < 1:
                user_lean = row['vote_share']
            count+=1
            state = row['state'].title()

    
    return user_lean, state
        

In [3]:
# csv file containing demographic data for each state
demo_data = pd.read_csv('state_demographics.csv')

# the specific variables we want to include
attributes = ['Age.Percent Under 5 Years', 'Age.Percent Under 18 Years',
           'Age.Percent 65 and Older', 'Miscellaneous.Percent Female',
           'Ethnicities.White Alone', 'Ethnicities.Black Alone', 
            'Ethnicities.Hispanic or Latino', 'Education.Bachelor\'s Degree or Higher']

# get demographic data from state name
def get_demographic_data(state):
    state_data = demo_data[demo_data['State'] == state].iloc[0][attributes]
    return state_data

In [4]:
from datetime import datetime, timedelta

# reformat the variable with a string containing the tweet date into a datetime object
def get_date_from_tweet(s):
    parts = s.split()
    datestring = parts[1] + ' ' + parts[2] + ' ' + parts[-1]
    return datetime.strptime(datestring, '%b %d %Y')

In [5]:
# csv file containing daily vaccine data from US states
vaccine_data = pd.read_csv('us_state_vaccinations.csv')

# variable we want to keep from day of tweet
vax_vars = ['people_vaccinated_per_hundred', 'daily_vaccinations_per_million']
# variable for daily vax rate
vax_per_day = 'daily_vaccinations_per_million'

def get_vaccine_data(state, time):
    # because my life was too easy
    if state == 'New York':
        state = 'New York State'
        
    state_data = vaccine_data[vaccine_data['location'] == state]
    
    # data for date of tweet
    today = state_data[state_data['date'] == time.strftime('%Y-%m-%d')].iloc[0][vax_vars]
        
    
    # one week before
    week_prior_time = time - timedelta(days=7)
    week_prior = state_data[state_data['date'] == week_prior_time.strftime('%Y-%m-%d')].iloc[0][vax_per_day]
    
    # one week after
    week_future_time = time + timedelta(days=7)
    week_future = state_data[state_data['date'] == week_future_time.strftime('%Y-%m-%d')].iloc[0][vax_per_day]
    
    # two weeks after
    two_week_future_time = time + timedelta(days=14)
    two_week_future = state_data[state_data['date'] == two_week_future_time.strftime('%Y-%m-%d')].iloc[0][vax_per_day]
    
    # reformat and return
    collected = *today , week_prior ,week_future,  two_week_future
    return pd.Series(collected)
    

In [6]:
# csv containing daily vaccination data for a bunch of different countries
c_data = pd.read_csv('country_vaccinations.csv')
# only keep US data
c_data = c_data[c_data['iso_code'] == 'USA']

# variable we want to keep from day of tweet
c_vax_vars = ['people_vaccinated_per_hundred', 'daily_vaccinations_per_million']
# variable for daily vax rate
c_vax_per_day = 'daily_vaccinations_per_million'

def get_national_data(time):
    
    # data for date of tweet
    today = c_data[c_data['date'] == time.strftime('%Y-%m-%d')].iloc[0][c_vax_vars]
        
    
    # one week before
    week_prior_time = time - timedelta(days=7)
    week_prior = c_data[c_data['date'] == week_prior_time.strftime('%Y-%m-%d')].iloc[0][c_vax_per_day]
    
    # one week after
    week_future_time = time + timedelta(days=7)
    week_future = c_data[c_data['date'] == week_future_time.strftime('%Y-%m-%d')].iloc[0][c_vax_per_day]
    
    # two weeks after
    two_week_future_time = time + timedelta(days=14)
    two_week_future = c_data[c_data['date'] == two_week_future_time.strftime('%Y-%m-%d')].iloc[0][c_vax_per_day]
    
    # reformat and return
    collected = *today, week_prior, week_future, two_week_future
    return pd.Series(collected)
    

In [7]:
# date when vaccine lottery was announced in each state
lottery_start_dates = {'Arkansas' : datetime(2021, 5, 25),
                       'California' : datetime(2021, 5, 27),
                       'Kentucky' : datetime(2021, 6, 4),
                       'Maryland' : datetime(2021, 5, 20),
                       'New Mexico' : datetime(2021, 6, 1),
                       'New York' : datetime(2021, 5, 20),
                       'Ohio' : datetime(2021, 5, 12),
                       'Oregon' : datetime(2021, 5, 21),
                       'Washington': datetime(2021, 6, 3),
                       'West Virginia': datetime(2021, 5, 27)
                      }


# check if tweet occured within 30 days of announcement of vaccine lottery
def during_lottery(state, time):
    if state not in lottery_start_dates.keys():
        return 0
    
    return (time - lottery_start_dates[state] > timedelta(days = 0)) and (time - lottery_start_dates[state] < timedelta(days = 30))
    
    

In [8]:
# columns to be dropped at the end of processing
# the only used columns are created_at, id, place

dropped = ['coordinates', 'created_at', 'hashtags', 'media', 'urls',
       'favorite_count', 'id', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'lang', 'place',
       'possibly_sensitive', 'quote_id', 'retweet_count', 'retweet_id',
       'retweet_screen_name', 'source', 'text', 'tweet_url', 'user_created_at',
       'user_id', 'user_default_profile_image', 'user_description',
       'user_favourites_count', 'user_followers_count', 'user_friends_count',
       'user_listed_count', 'user_location', 'user_name', 'user_screen_name',
       'user_statuses_count', 'user_time_zone', 'user_urls', 'user_verified']

def process_csv(filename):
    df = pd.read_csv(os.path.join('data_hydrated', filename))
    if 'may' in filename:
        directory = 'data/2021_05'
    else:
        directory = 'data/2021_06'
    sentiments = pd.read_csv(os.path.join(directory, filename), header = None)
    sentiments.columns = ['id', 'sentiment']

    # add sentiments back into hydrated DataFrame
    df_with_sentiments = pd.merge(df, sentiments, on=['id'], how='inner')
    
    # add partisan lean and state name to DataFrame
    df_with_sentiments[['partisan_lean', 'state']] = df_with_sentiments['place'].apply(lambda x: pd.Series(get_partisan_lean(x)))
    
    # remove non US and non-identifiable entries
    df_us = df_with_sentiments[df_with_sentiments['partisan_lean'] != -1]
    
    # add demographic data to tweets
    df_us[attributes] = df_us['state'].apply(get_demographic_data)
    
    # add python datetime to tweets
    df_us['date'] = df_us['created_at'].apply(get_date_from_tweet)
    
    # add state level vaccine data
    df_us[['state percent vaccinated', 'state_rate_0', 'state_rate-7', 'state_rate+7', 'state_rate+14']] = df_us.apply(lambda row: get_vaccine_data(row['state'], row['date']), axis=1)
    
    # add national level vaccine data
    df_us[['US percent vaccinated', 'US_rate_0', 'US_rate-7', 'US_rate+7', 'US_rate+14']] = df_us['date'].apply(get_national_data)
    
    df_us['lottery'] = df_us.apply(lambda row: during_lottery(row['state'], row['date']), axis = 1)
    
    df_us = df_us.drop(columns = dropped)
    return df_us
    


In [9]:
df_list = []

for filename in os.listdir('data_hydrated'):
    print(filename)
    df_list.append(process_csv(filename))

2021_june24_june25.csv
2021_may23_may24.csv
2021_may1_may2.csv
2021_june29_june30.csv
2021_june21_june22.csv
2021_may13_may14.csv
2021_june27_june28.csv
2021_june5_june6.csv
2021_june9_june10.csv
2021_may15_may16.csv
2021_june10_june11.csv
2021_june3_june4.csv
2021_june15_june16.csv
2021_may25_may26.csv
2021_june2_june3.csv
2021_may17_may18.csv
2021_june28_june29.csv
2021_june4_june5.csv
2021_june19_june20.csv
2021_may27_may28.csv
2021_may6_may7.csv
2021_june26_june27.csv
2021_may22_may23.csv
2021_may24_may25.csv
2021_may30_may31.csv
2021_may31_june1.csv
2021_may29_may30.csv
2021_may14_may15.csv
2021_may8_may9.csv
2021_june23_june24.csv
2021_june7_june8.csv
2021_may12_may13.csv
2021_june12_june13.csv
2021_june14_june15.csv
2021_june1_june2.csv
2021_june17_june18.csv
2021_may3_may4.csv
2021_june11_june12.csv
2021_may21_may22.csv
2021_may19_may20.csv
2021_june20_june21.csv
2021_may2_may3.csv
2021_may7_may8.csv
2021_june25_june26.csv
2021_may11_may12.csv
2021_june30_july1.csv
2021_june18_

In [10]:
df = pd.concat(df_list)

df.shape

df.to_csv('full_data.csv')

In [11]:
df[df['lottery'] == 1].shape, df[df['lottery'] == 0].shape

((1420, 23), (6193, 23))